# Problem 1 (15 points)
Let $S(T)=S(0)\exp{(\sigma\sqrt{T}Z+(r-\frac{1}{2}\sigma^2)T)}$ be the final stock price value in the Black-Scholes model and $Z\sim\mathcal{N}(0, 1)$.  
Set $$\theta=E^*[(K-S(T))^+].$$ Then the arbitrage free initial price for this put option is $e^{-rT}\theta$.

(a). Estimate the value of $\theta$ using standard Monte Carlo simulation. 

In [1]:
import numpy as np

In [2]:
# Write a function to estimate theta and estimate the variance of the \hat{theta}_n
# using Monte Carlo simulation with sample size n

def MC_est(T, r, sig, S0, K, n, seed):
    """
    input: 
        T: Maturity Time
        r: Risk-free interest rate
        sig: volatility
        S0: Current stock price
        K: Strike price
        n: sample size of Monte Carlo simulation
        seed: random seed
    -------------------------------
    output: (a 1-dim numpy array) -- similar as np.numpy([est_theta, est_var])
            first element in the array:
                estimation on the value of E^*[(K-S(T))^+] for European put option;
            second element in the array:
                estimation on the variance of the estimator
            
    """
    np.random.seed(seed)
    # YOUR CODE HERE
    Z = np.random.randn(n)
    S_t = S0*np.exp(sig*np.sqrt(T)*Z+(r-0.5*sig**2)*T)
    x = np.maximum((K - S_t), 0)
    est_theta = np.mean(x)
    var_theta = np.var(x,ddof = 1)/n
    
    return np.array([est_theta,var_theta])

In [3]:
"""Check your estimation on the value of theta, i.e. \hat{\theta}_n"""
assert round(MC_est(2, 0.01, 0.2, 5, 6, 1000, 123)[0], 2) == 1.21


In [4]:
"""Check your estimation on the variance of \hat{theta}_n"""
assert round(MC_est(2, 0.01, 0.2, 5, 6, 1000, 123)[1], 5) == 0.00102


(b) Use the method of antithetic variables to estimate $\theta$. Write a function to estimate $\theta$ (calling the estimator $\hat{\theta}_{av}$). Also estimate the variance of $\hat{\theta}_{av}$. 

Rewrite the estimator as $\theta = \mathbb{E}^*[g(Z)]$.  The next function you write will be essentially $g$.  Once you find what $g$ is, show that $g$ is a decreasing function on paper and turn it via Gradescope (hint: taking derivative, we have two regions since $(K-S(t))^+$).

In [5]:
# Generate antithetic variables (Z, -Z) for computing E[g(Z)] step by step
# Set a set of parameter first
T = 2
r = 0.01
sig = 0.2
S0 = 5
K = 6
n = 1000

Here are the steps that we will use to create the antithetic variable estimator:

1. generate int(n/2) samples to form Z1
2. write g function to simplify our steps later
3. compute g(Z_anti) as  $\frac{g(Z1) + g(-Z1)}{2}$
4. calculate AV_est_theta (estimating with antithetic variables) and AV_var_theta (estimating the variance of the estimator); AV is short for "antithetic variable".

In [6]:
np.random.seed(123) # Do not modify the seed here!!!
# Do Step 1 here in one line
# YOUR CODE HERE
Z1 = np.random.randn(int(n/2))

In [7]:
# Do Step 2 here
# write the g as function taking parameters: x, K, sig, r, T, S0 here

def g(x, K, sig, r, T, S0):
    """
    output: 
        function value: g(x)
    """
    # YOUR CODE HERE
    
    S_t = S0*np.exp(sig*np.sqrt(T)*x+(r-0.5*sig**2)*T)
    x = np.maximum((K - S_t), 0)
    
    return x


In [8]:
"""Check your g function"""
assert round(g(0, K, sig, r, T, S0), 2) == 1.10

In [9]:
# Do Step 3 here: Calculate gZ_anti by calling the function g
# YOUR CODE HERE
gZ_anti = 0.5*(g(Z1, K, sig, r, T, S0)+g(-Z1, K, sig, r, T, S0))

In [10]:
# Do Step 4 here: AV_est_theta (estimated mean); AV_var_theta (estimated variance of the estimator)
# YOUR CODE HERE
AV_est_theta = np.mean(gZ_anti)
AV_var_theta = np.var(gZ_anti,ddof = 1)/len(gZ_anti)

In [11]:
"""Check your anwser here"""
assert round(AV_est_theta,2) == 1.18
assert round(AV_var_theta,5) == 7e-05

In [12]:
# Use the code from the above cell to write a funtion in preparation for part c
def AV_est(T, r, sig, S0, K, n, seed):
    """Estimating E(K-ST)^+ using antithetic variables
    input: 
        T: Maturity Time
        r: Risk-free interest rate
        sig: volatility
        S0: Current stock price
        K: Strike price
        n: total sample size (note: for antithetic variables, we generate int(n/2) samples and then take -Z)
        seed: random seed
    -------------------------------
    output: (a 1-dim numpy array)
            first element in the array:
                estimation on the value of E^*[(K-S(T))^+] using antithetic variables
            second element in the array:
                estimation on the variance of the estimator
    """
    np.random.seed(seed)
    # YOUR CODE HERE
    Z1 = np.random.randn(int(n/2))
    gZ_anti = 0.5*(g(Z1, K, sig, r, T, S0)+g(-Z1, K, sig, r, T, S0))
    AV_est_theta = np.mean(gZ_anti)
    AV_var_theta = np.var(gZ_anti,ddof = 1)/len(gZ_anti)
    
    return np.array([AV_est_theta,AV_var_theta])
    

In [13]:
"""Check your answer here"""
assert round(AV_est(2, 0.01, 0.2, 5, 6, 1000, 123)[0], 3) == 1.176
assert round(AV_est(2, 0.01, 0.2, 5, 6, 1000, 123)[1], 5) == 7e-05

(c) Apply the functions for $T=5, r=0.02, \sigma=0.15, S(0)=5, K=6$ and with $n=100, 500, 1000, 5000$ samples. Report your results in a table. Comment on the relationship between $\hat{\theta}_n$ and $\hat{\theta}_{av}$ and the relationship between their estimated variances. 

In [14]:
# import pandas as pd and create a dataframe called p1_result
# the dataframe is with columns ['sample size', 'theta_n', 'theta_av', 'theta_n_var', 'theta_av_var']
# sample size: [100, 500, 1000, 5000]

# Create the dataframe and fill in the 'sample size' column with [100, 500, 1000, 5000]
import pandas as pd

# YOUR CODE HERE
T=5
r=0.02
sig = 0.15
S0=5
K=6
p1_result = pd.DataFrame(columns = ['sample size', 'theta_n', 'theta_av', 'theta_n_var', 'theta_av_var'])
p1_result['sample size']  = [100, 500, 1000, 5000]
p1_result

,sample size,theta_n,theta_av,theta_n_var,theta_av_var
0,100,NaN,NaN,NaN,NaN
1,500,NaN,NaN,NaN,NaN
2,1000,NaN,NaN,NaN,NaN
3,5000,NaN,NaN,NaN,NaN


In [15]:
seed = 5 # Fix the seed = 5 DO NOT CHANGE

"""Fill the rest of the result table now
theta_n: estimation using standard MC
theta_av: estimation using MC with antithetic variables(AV)
theta_n_var: estimated variance for the standard MC
theta_av_var: estimated variance for the MC with AV"""

# Hints:
# theta_n: apply MC_est; the first element in the output of MC_est
# theta_av: apply AV_est; the first element in the output of AV_est
# theta_n_var: apply MC_est; the second element in the output of MC_est
# theta_av_var: apply AV_est; the second element in the output of AV_est

# YOUR CODE HERE
for i in p1_result.index:
    size = p1_result.loc[i,'sample size']
    p1_result.loc[i,'theta_n'] = MC_est(T, r, sig, S0, K, size, seed)[0]
    p1_result.loc[i,'theta_av'] = AV_est(T, r, sig, S0, K, size, seed)[0]
    p1_result.loc[i,'theta_n_var'] = MC_est(T, r, sig, S0, K, size, seed)[1]
    p1_result.loc[i,'theta_av_var'] = AV_est(T, r, sig, S0, K, size, seed)[1]

In [16]:
# print the result
# YOUR CODE HERE
p1_result

   sample size   theta_n  theta_av theta_n_var theta_av_var
0          100  0.937635  0.959436    0.008773     0.001579
1          500  1.016169  1.008538    0.002096     0.000365
2         1000  0.987074  1.020911    0.001071      0.00018
3         5000  1.031489  1.026406     0.00023     0.000038


In [17]:
assert round(p1_result[p1_result['sample size'] == 100].theta_n.values[0], 2) == 0.94

In [18]:
assert round(p1_result[p1_result['sample size'] == 500].theta_av.values[0], 2) == 1.01

In [19]:
assert round(p1_result[p1_result['sample size'] == 1000].theta_n_var.values[0], 3) == 0.001

In [20]:
assert round(p1_result[p1_result['sample size'] == 5000].theta_av_var.values[0], 6) == 3.8e-05

In the cell, below, comment on the relationship between $\hat{\theta}_n$ and $\hat{\theta}_{av}$.  

In the cell, below, comment on the relationship between $\hat{\theta}_n$ and $\hat{\theta}_{av}$.  

# Problem 2 (17 points)
The goal is to compare the performance in computing the initial price for a European derivative using importance sampling versus standard Monte Carlo sampling.

Consider $g(Z)=20\times\mathbf{1}_{\{Z\geq 3\}}$, where $Z\sim\mathcal{N}(0,1)$. Suppose the inital price for this option is given by $$\theta=20 E^*[\mathbf{1}_{\{Z\geq 3\}}].$$

(a) Draw all graphs as required in the homework by hand. (submit this part in Gradescope)

(b) Write a function to estimate $\theta$ using standard Monte Carlo simulation, to estimate variance of $\hat{\theta}_n^f$ using $\hat{\sigma}_n^{2,f}=\frac{1}{n-1}\sum_{i=1}^n(g(Z_i-\hat{\theta}_n^f)^2$ where $\{Z_i\}_{i=1}^n$ are i.i.d. samples for $Z$. (5 points)

In [21]:
# generate Z = np.random.normal(0, 1, 100) and then assign gZ = g(Z)
np.random.seed(1)
# YOUR CODE HERE
Z = np.random.normal(0, 1, 100)

gZ = np.array([20 if z >=3 else 0 for z in Z])

In [22]:
assert sum(gZ) == 0

In [23]:
# write a function called MC_est_digital to estimate \theta and compute variance of the the estimator
def MC_est_digital(n, seed):
    """
    input: 
        n: Monte Carlo sample size 
        seed: random seed
    output: (a 1-dim numpy array) 
        EX_g: estimation of theta
        var_g: estimated variance of the estimator \hat{theta}_n
    """
    np.random.seed(seed)
    # YOUR CODE HERE
    Z = np.random.normal(0, 1, n)
    gZ = np.array([20 if z >=3 else 0 for z in Z])
    EX_g = np.mean(gZ)
    var_g = np.var(gZ,ddof = 1)/n
    
    return np.array([EX_g,var_g])

In [24]:
assert MC_est_digital(1000, 123)[0] == 0.02
assert MC_est_digital(1000, 123)[1] == 0.0004

(c) Write another function called *IMP_est* to estimate $\theta$ using a Monte Carlo importance sampling method. 

In [25]:
# generate Zmu (including 100 data points) from the normal(4, 1) distribution and compute kZ = k(Z)
np.random.seed(3)
# YOUR CODE HERE
mu = 3
Zmu = np.random.normal(mu, 1, 100)
kZ = np.array([20 if z >=3 else 0 for z in Zmu])*np.exp(-mu*Zmu+0.5*(mu**2))
sum(kZ)

1.930055194698274

In [26]:
assert round(sum(kZ), 2) == 1.93

In [31]:
# write a function called IMP_est_digital to estimate \theta and compute variance of the the estimator 
# using importance sampling
def IMP_est_digital(n, seed): 
    """
    input: 
        n: Monte Carlo sample size 
        seed: random seed
    output: (a 1-dim numpy array) 
        imp_EX_k: estimation of theta
        imp_var_k: estimated variance of the estimator \hat{theta}_n
    """
    np.random.seed(seed)
    # YOUR CODE HERE
    mu = 3
    Zmu = np.random.normal(mu, 1, n)
    kZ = np.array([20 if z >=3 else 0 for z in Zmu])*np.exp(-mu*Zmu+0.5*(mu**2))
    imp_EX_k = np.mean(kZ)
    imp_var_k = np.var(kZ,ddof = 1)/n
    
    return np.array([imp_EX_k,imp_var_k])

IMP_est_digital(1000, 123)

array([2.67270676e-02, 2.54133379e-06])

In [28]:
assert round(IMP_est_digital(1000, 123)[0], 6) == 0.032283
assert round(IMP_est_digital(1000, 123)[1], 11) == 3.64872e-06

AssertionError: 

(d) Run the functions (*MC_est_digital*, *IMP_est_digital*) above for $n=100, 500, 1000$. Compare the estimated values of $\theta$ and their variances for the two methods. Comment on your results. Notice the difference in estimated initial prices if one were to sell a million of these digital options at time zero.

In [ ]:
# Similarly, create pandas dataframe: p2_result 
# columns=['sample size', 'theta', 'est_var', 'imp_theta', 'imp_var']
# fill in the 'sample size' column with np.array([100, 500, 1000], int)

# YOUR CODE HERE
p2_result = pd.DataFrame(columns = ['sample size', 'theta', 'est_var', 'imp_theta', 'imp_var'])
p2_result['sample size']  = [100, 500, 1000, 5000]
p2_result

In [ ]:
seed = 5 # SET A SEED; DO NOT CHANGE

# fill in the columns using functions: MC_est_digital, IMP_est_digital
# theta: first output of MC_est_digital
# est_var: second output of MC_est_digital
# imp_theta: first output of IMP_est_digital
# imp_var: second output of IMP_est_digital

# YOUR CODE HERE
for i in p2_result.index:
    size = p2_result.loc[i,'sample size']
    p2_result.loc[i,'theta'] = MC_est_digital(size, seed)[0]
    p2_result.loc[i,'est_var'] = MC_est_digital(size, seed)[1]
    p2_result.loc[i,'imp_theta'] = IMP_est_digital(size, seed)[0]
    p2_result.loc[i,'imp_var'] = IMP_est_digital(size, seed)[1]

In [ ]:
# print your p2_result
p2_result

In [ ]:
assert round(p2_result[p2_result['sample size'] == 100].imp_theta.values[0], 4) == 0.0385
assert round(p2_result[p2_result['sample size'] == 100].imp_var.values[0], 10) == 3.80045e-05

Comment on your results in the cell below. 

YOUR ANSWER HERE